# Analyze Promotion Effect

In [1]:
# load in packages
from itertools import combinations
from test_results import score
import numpy as np
import pandas as pd
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

In [2]:
# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84534 entries, 0 to 84533
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         84534 non-null  int64  
 1   Promotion  84534 non-null  object 
 2   purchase   84534 non-null  int64  
 3   V1         84534 non-null  int64  
 4   V2         84534 non-null  float64
 5   V3         84534 non-null  float64
 6   V4         84534 non-null  int64  
 7   V5         84534 non-null  int64  
 8   V6         84534 non-null  int64  
 9   V7         84534 non-null  int64  
dtypes: float64(2), int64(7), object(1)
memory usage: 6.4+ MB


## Confirm randomized control/treatment groups

First, we'll check whether there's sufficient evidence the trial is truly randomized by testing that customers were assigned to the treatment group (those that received the promotion) or control group (those that didn't) with equal probability.

In [4]:
# 0-1 encoding promotion
train_data['Promotion'] = train_data['Promotion'].apply(lambda x: 1 if x == 'Yes' else 0)

First we'll find a bootstrapped (non-parametric) confidence interval for the proportion of customers in the treatment group.

In [5]:
def boot_prop_ci(data, q, c = .95, n_trials = 1000):
    """
    Compute a confidence interval for proportion assigned to promotion using bootstrap
    method.
    
    Input parameters:
        data: 0/1 data in form of 1-D array-like (e.g. numpy array or Pandas series)
        c: confidence interval width
        n_trials: number of bootstrap samples to perform
    
    Output value:
        ci: Tuple indicating lower and upper bounds of bootstrapped
            confidence interval
    """
    
    # initialize storage of bootstrapped sample quantiles
    n_points = len(data)
    sample_ps = []
    
    # For each trial...
    for _ in range(n_trials):
        # draw a random sample from the data with replacement...
        sample = np.random.choice(data, size=n_points, replace=True)
        # compute the desired proportion and add the value to the list of sampled quantiles
        sample_ps.append(sample.mean())
        
    # Compute the confidence interval bounds
    lower_limit = np.quantile(sample_ps, (1 - c)/2)
    upper_limit = np.quantile(sample_ps, 1 - (1 - c)/2)
    
    return (lower_limit, upper_limit)

In [6]:
# 99.9% bootstrapped confidence interval for proportion assigned to promotion
boot_prop_ci(train_data['Promotion'], 0.5, c = .999, n_trials = 1000)

(0.49527305581186265, 0.5059503513379232)

This is very high confidence that the proportion assigned to promotion was $\approx 0.5$.

Just for fun (and thoroughness!) we'll contruction a Bayesian MAP estimate and credibility interval for the treatment proportion.

In [7]:
# 95% bayesian credibility interval for proportion assigned to promotion
from scipy.stats import bayes_mvs

results = bayes_mvs(train_data['Promotion'], alpha=0.999)
print(f'estimate: {results[0].statistic}')
print(f'CI: {results[0].minmax}')

estimate: 0.5011474672912675
CI: (0.4954887424819059, 0.506806192100629)


## Test for treatment effect

Now we'll examine statistical evidence for a treatment effect, that is, that the treatment group are more likely to purchase than the control group.

First we'll do a classic parametric Z-test for difference of proportions of customers who purchased in the treatment and control groups. Note that this is the same as the incremental response rate (IRR).

In [8]:
# parametric hypothesis test for difference of proportions
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
k0, k1, n0, n1 = train_data.groupby(['Promotion'])['purchase'].value_counts()
stat, pval = proportions_ztest([k0, k1], [n0, n1], value=0.5)
print(f'p-value is {pval}')

p-value is 2.5162768110176954e-134


This is a highly significant result. Since parametric test may introduce unwanted assumptions, and since $p$-values have their detractors, let's do a bootstrapped confidence interval.

In [9]:
def boot_diff_prop_ci(data, c = .95, n_trials = 1000):
    """
    Compute a confidence interval for difference in purchase proportions between
    promotion groups.
    
    Input parameters:
        data: 0/1 data in form of 1-D array-like (e.g. numpy array or Pandas series)
        c: confidence interval width
        n_trials: number of bootstrap samples to perform
    
    Output value:
        ci: Tuple indicating lower and upper bounds of bootstrapped
            confidence interval
    """
    # initialize storage of bootstrapped sample quantiles
    sample_diffs = []
    
    # For each trial...
    for _ in range(n_trials):
        # draw a random sample from the data with replacement...
        sample = data.sample(n=data.shape[0], replace=True, axis=0)
        # compute the difference of proportions that purchased
        p0, p1 = sample.groupby(['Promotion'])['purchase'].mean()
        sample_diffs.append(p1 - p0)
        
    # Compute the confidence interval bounds
    lower_limit = np.quantile(sample_diffs, (1 - c)/2)
    upper_limit = np.quantile(sample_diffs, 1 - (1 - c)/2)
    
    return (lower_limit, upper_limit)

In [10]:
# bootstrapped 99% confidence interval for difference of proportions
boot_diff_prop_ci(train_data, c = .99, n_trials = 1000)

(0.007661851261916283, 0.01150491098228835)

With 99% confidence, the treatment appears to have had an effect.

## Indentify effects of treatment

Let's look at the effect of the promotion.

In [11]:
train_data['Promotion'] = train_data['Promotion'].apply(lambda x: "Yes" if x == 1 else "No")
irr, nir = score(train_data)
print(f'The incremental response rate (IRR) for the training data is:\n{irr}')
print(f'\nThe net incremental revenue (NIR) for the training data is:\n{nir}')

The incremental response rate (IRR) for the training data is:
0.009454547819772702

The net incremental revenue (NIR) for the training data is:
-2334.5999999999995


Even though the experiment shows $\approx 0.9\%$ increase in response rate, the net incremental response was negative, meaning revenue from the treatment group was less than from the control group. The experiment had a small positive effect but it doesn't appear to be enough to justify its cost.